In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import tree
from IPython import get_ipython
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('Crop_recommendation.csv')

In [3]:
data.head()

,Nitrogen,phosphorus,potassium,temperature,humidity,ph,rainfall,label,Unnamed: 8,Unnamed: 9
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,NaN,NaN
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice,NaN,NaN
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice,NaN,NaN
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice,NaN,NaN
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice,NaN,NaN


In [4]:
data.tail()

,Nitrogen,phosphorus,potassium,temperature,humidity,ph,rainfall,label,Unnamed: 8,Unnamed: 9
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,coffee,NaN,NaN
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,coffee,NaN,NaN
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,coffee,NaN,NaN
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,coffee,NaN,NaN
2199,104,18,30,23.603016,60.396475,6.779833,140.937041,coffee,NaN,NaN


In [5]:
data.shape

(2200, 10)

In [6]:
data.columns

Index(['Nitrogen', 'phosphorus', 'potassium', 'temperature', 'humidity', 'ph',
       'rainfall', 'label', 'Unnamed: 8', 'Unnamed: 9'],
      dtype='object')

In [7]:
data.duplicated().sum()

0

In [8]:
data.isnull().sum()

Nitrogen          0
phosphorus        0
potassium         0
temperature       0
humidity          0
ph                0
rainfall          0
label             0
Unnamed: 8     2200
Unnamed: 9     2200
dtype: int64

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Nitrogen     2200 non-null   int64  
 1   phosphorus   2200 non-null   int64  
 2   potassium    2200 non-null   int64  
 3   temperature  2200 non-null   float64
 4   humidity     2200 non-null   float64
 5   ph           2200 non-null   float64
 6   rainfall     2200 non-null   float64
 7   label        2200 non-null   object 
 8   Unnamed: 8   0 non-null      float64
 9   Unnamed: 9   0 non-null      float64
dtypes: float64(6), int64(3), object(1)
memory usage: 172.0+ KB


In [10]:
data.describe()

,Nitrogen,phosphorus,potassium,temperature,humidity,ph,rainfall,Unnamed: 8,Unnamed: 9
count,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,0.0,0.0
mean,50.551818,53.362727,48.149091,25.616244,71.481779,6.469480,103.463655,NaN,NaN
std,36.917334,32.985883,50.647931,5.063749,22.263812,0.773938,54.958389,NaN,NaN
min,0.000000,5.000000,5.000000,8.825675,14.258040,3.504752,20.211267,NaN,NaN
25%,21.000000,28.000000,20.000000,22.769375,60.261953,5.971693,64.551686,NaN,NaN
50%,37.000000,51.000000,32.000000,25.598693,80.473146,6.425045,94.867624,NaN,NaN
75%,84.250000,68.000000,49.000000,28.561654,89.948771,6.923643,124.267508,NaN,NaN
max,140.000000,145.000000,205.000000,43.675493,99.981876,9.935091,298.560117,NaN,NaN


In [11]:
data.nunique()

Nitrogen        137
phosphorus      117
potassium        73
temperature    2200
humidity       2200
ph             2200
rainfall       2200
label            22
Unnamed: 8        0
Unnamed: 9        0
dtype: int64

In [12]:
data['label'].unique()

array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee'],
      dtype=object)

In [13]:
data['label'].value_counts()

rice           100
maize          100
jute           100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
muskmelon      100
watermelon     100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
blackgram      100
mungbean       100
mothbeans      100
pigeonpeas     100
kidneybeans    100
chickpea       100
coffee         100
Name: label, dtype: int64

In [14]:
crop_summary = pd.pivot_table(data,index=['label'], aggfunc='mean')

In [15]:
crop_summary

,Nitrogen,humidity,ph,phosphorus,potassium,rainfall,temperature
label,,,,,,,
apple,20.80,92.333383,5.929663,134.22,199.89,112.654779,22.630942
banana,100.23,80.358123,5.983893,82.01,50.05,104.626980,27.376798
blackgram,40.02,65.118426,7.133952,67.47,19.24,67.884151,29.973340
chickpea,40.09,16.860439,7.336957,67.79,79.92,80.058977,18.872847
coconut,21.98,94.844272,5.976562,16.93,30.59,175.686646,27.409892
coffee,101.20,58.869846,6.790308,28.74,29.94,158.066295,25.540477
cotton,117.77,79.843474,6.912675,46.24,19.56,80.398043,23.988958
grapes,23.18,81.875228,6.025937,132.53,200.11,69.611829,23.849575
jute,78.40,79.639864,6.732778,46.86,39.99,174.792798,24.958376


In [16]:
x = data.drop('label', axis=1)
y = data['label']

DECISION TREE MODEL

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, shuffle = True)

In [19]:
param_dist = {"max_depth": [3, None],
              "min_samples_leaf": (1,9),
              "criterion": ["gini", "entropy"],
               "random_state": [0, 100]}

In [20]:
from sklearn.tree import DecisionTreeClassifier
classifier_dt= DecisionTreeClassifier()
classifier_dtt = RandomizedSearchCV(classifier_dt, param_dist, cv=5)
classifier_dtt.fit(X_train, y_train)

ValueError: 
All the 50 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\femibewaji\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\femibewaji\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 889, in fit
    super().fit(
  File "C:\Users\femibewaji\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 186, in fit
    X, y = self._validate_data(
  File "C:\Users\femibewaji\anaconda3\lib\site-packages\sklearn\base.py", line 560, in _validate_data
    X = check_array(X, input_name="X", **check_X_params)
  File "C:\Users\femibewaji\anaconda3\lib\site-packages\sklearn\

In [ ]:
RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, None],
                                        'min_samples_leaf': (1, 9),
                                        'random_state': [0, 100]})

In [ ]:
y_pred=classifier_dtt.predict(X_test)

In [ ]:
accuracy=accuracy_score(y_pred, y_test)
#acc.append(accuracy)
#model.append('Decision Tree')
print('Decision Tree Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test,y_pred)))
print('Tuned Parameters:{}'.format(classifier_dtt.best_params_))

Decision Tree Model accuracy score: 0.9924
Tuned Parameters:{'random_state': 100, 'min_samples_leaf': 1, 'max_depth': None, 'criterion': 'gini'}

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        29
      banana       1.00      1.00      1.00        37
   blackgram       0.97      1.00      0.98        28
    chickpea       1.00      1.00      1.00        19
     coconut       1.00      1.00      1.00        31
      coffee       1.00      0.97      0.98        32
      cotton       1.00      1.00      1.00        31
      grapes       1.00      1.00      1.00        28
        jute       0.92      0.96      0.94        25
 kidneybeans       1.00      1.00      1.00        32
      lentil       1.00      1.00      1.00        40
       maize       0.97      1.00      0.98        31
       mango       1.00      1.00      1.00        32
   mothbeans       1.00      0.96      0.98        23
    mungbean       1.00      1.00      1.00        32
   muskmelon       1.00      1.00      1.00        25
      orange       1.00      1.00      1.00        42
      papaya       1.00      1.00      1.00        27
  pigeonpeas       1.00      1.00      1.00        33
 pomegranate       1.00      1.00      1.00        32
        rice       0.97      0.93      0.95        30
  watermelon       1.00      1.00      1.00        21

    accuracy                           0.99       660
   macro avg       0.99      0.99      0.99       660
weighted avg       0.99      0.99      0.99       660

In [ ]:
x = data.drop('label', axis=1)
y = data['label']

LOGISTIC REGRESSION

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, shuffle = True)

In [ ]:
param_dist = {"penalty": ('l1','l2','elsaticnet','none'),
               "random_state": [0, 100],
              "max_iter": (0,100)}

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier_lr = LogisticRegression()
classifier_lrr = RandomizedSearchCV(classifier_lr, param_dist, cv=5)
classifier_lrr.fit(X_train, y_train)

In [ ]:
RandomizedSearchCV(cv=5, estimator=LogisticRegression(),
                   param_distributions={'max_iter': (0, 100),
                                        'penalty': ('l1', 'l2', 'elsaticnet',
                                                    'none'),
                                        'random_state': [0, 100]})

In [ ]:
y_pred = classifier_lrr.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
#acc.append(accuracy)
#model.append('Logistic Regression')
print('Logistic Regression Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test,y_pred)))
print('Tuned Parameters:{}'.format(classifier_lrr.best_params_))

Logistic Regression Model accuracy score: 0.9614
Tuned Parameters:{'random_state': 100, 'penalty': 'l2', 'max_iter': 100}

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        20
      banana       1.00      1.00      1.00        26
   blackgram       0.90      0.90      0.90        20
    chickpea       1.00      1.00      1.00        23
     coconut       1.00      1.00      1.00        22
      coffee       1.00      1.00      1.00        15
      cotton       0.95      0.95      0.95        21
      grapes       1.00      1.00      1.00        19
        jute       0.90      1.00      0.95        19
 kidneybeans       1.00      1.00      1.00        21
      lentil       0.79      0.94      0.86        16
       maize       0.88      0.88      0.88        17
       mango       1.00      1.00      1.00        19
   mothbeans       0.78      0.78      0.78        18
    mungbean       0.93      0.88      0.90        16
   muskmelon       1.00      1.00      1.00        26
      orange       1.00      1.00      1.00        19
      papaya       1.00      0.88      0.94        25
  pigeonpeas       0.92      1.00      0.96        12
 pomegranate       1.00      1.00      1.00        26
        rice       1.00      0.91      0.95        23
  watermelon       1.00      1.00      1.00        17

    accuracy                           0.96       440
   macro avg       0.96      0.96      0.96       440
weighted avg       0.96      0.96      0.96       440

In [ ]:
x = data.drop('label', axis=1)
y = data['label']

RANDOM FOREST CLASSIFIER

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.10, shuffle = True)

In [ ]:
param_dist = {"n_estimators": (0, 100),
               "random_state": (0, 100),
              "criterion": ['gini', 'entropy', 'log_loss'],
              "max_depth": (0,100),
              "max_features": ['sqrt','log2','none']}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier_rf= RandomForestClassifier()
classifier_rff = RandomizedSearchCV(classifier_rf, param_dist, cv=5)
classifier_rff.fit(X_train, y_train)

In [ ]:
RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': (0, 100),
                                        'max_features': ['sqrt', 'log2',
                                                         'none'],
                                        'n_estimators': (0, 100),
                                        'random_state': (0, 100)})

In [ ]:
y_pred= classifier_rff.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
#acc.append(accuracy)
#model.append('Random Forest')
print('Random Forest Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test,y_pred)))
print('Tuned Parameters:{}'.format(classifier_rff.best_params_))

Random Forest Model accuracy score: 0.9955
Tuned Parameters:{'random_state': 100, 'n_estimators': 100, 'max_features': 'sqrt', 'max_depth': 100, 'criterion': 'entropy'}

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        16
      banana       1.00      1.00      1.00         9
   blackgram       1.00      1.00      1.00         2
    chickpea       1.00      1.00      1.00         7
     coconut       1.00      1.00      1.00        11
      coffee       1.00      1.00      1.00         5
      cotton       1.00      1.00      1.00        13
      grapes       1.00      1.00      1.00        12
        jute       1.00      0.95      0.98        21
 kidneybeans       1.00      1.00      1.00        20
      lentil       1.00      1.00      1.00         7
       maize       1.00      1.00      1.00         8
       mango       1.00      1.00      1.00        11
   mothbeans       1.00      1.00      1.00         7
    mungbean       1.00      1.00      1.00         9
   muskmelon       1.00      1.00      1.00         8
      orange       1.00      1.00      1.00         9
      papaya       1.00      1.00      1.00         8
  pigeonpeas       1.00      1.00      1.00        11
 pomegranate       1.00      1.00      1.00         6
        rice       0.89      1.00      0.94         8
  watermelon       1.00      1.00      1.00        12

    accuracy                           1.00       220
   macro avg       0.99      1.00      1.00       220
weighted avg       1.00      1.00      1.00       220

In [ ]:
x = data.drop('label', axis=1)
y = data['label']

MLP CLASSIFIER

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.40, shuffle = True)

In [ ]:
param_dist = {"hidden_layer_sizes": (10, 10, 10),
               "max_iter": (0, 1000),
              "activation": ['identity', 'logistic', 'tanh', 'relu'],
              "solver": ['lbfgs','sgd', 'adam'],
              "learning_rate": ['constant', 'invscaling', 'adaptive']}

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
StandardScaler()
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
classifier_mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)
classifier_mlpp = RandomizedSearchCV(classifier_mlp, param_dist, cv=5)
classifier_mlpp.fit(X_train, y_train)
RandomizedSearchCV(cv=5,
                   estimator=MLPClassifier(hidden_layer_sizes=(10, 10, 10),
                                           max_iter=1000),
                   param_distributions={'activation': ['identity', 'logistic',
                                                       'tanh', 'relu'],
                                        'hidden_layer_sizes': (10, 10, 10),
                                        'learning_rate': ['constant',
                                                          'invscaling',
                                                          'adaptive'],
                                        'max_iter': (0, 1000),
                                        'solver': ['lbfgs', 'sgd', 'adam']})

In [ ]:
y_pred = classifier_mlpp.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
#acc.append(accuracy)
#model.append('MLP Classifier')
print('MLP Classifier accuracy score: {0:0.4f}'.format(accuracy_score(y_test,y_pred)))
print('Tuned Parameters:{}'.format(classifier_mlpp.best_params_))

MLP Classifier accuracy score: 0.9784
Tuned Parameters:{'solver': 'adam', 'max_iter': 1000, 'learning_rate': 'invscaling', 'hidden_layer_sizes': 10, 'activation': 'relu'}

In [ ]:
              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        44
      banana       1.00      1.00      1.00        41
   blackgram       0.94      1.00      0.97        44
    chickpea       1.00      1.00      1.00        44
     coconut       1.00      1.00      1.00        37
      coffee       0.97      1.00      0.99        35
      cotton       1.00      1.00      1.00        43
      grapes       1.00      1.00      1.00        33
        jute       0.82      0.85      0.84        39
 kidneybeans       0.98      1.00      0.99        43
      lentil       1.00      0.98      0.99        46
       maize       1.00      0.97      0.98        33
       mango       1.00      1.00      1.00        46
   mothbeans       0.97      0.95      0.96        39
    mungbean       1.00      1.00      1.00        40
   muskmelon       1.00      1.00      1.00        41
      orange       0.97      1.00      0.99        37
      papaya       1.00      1.00      1.00        41
  pigeonpeas       1.00      0.95      0.97        37
 pomegranate       1.00      1.00      1.00        45
        rice       0.85      0.80      0.82        35
  watermelon       1.00      1.00      1.00        37

    accuracy                           0.98       880
   macro avg       0.98      0.98      0.98       880
weighted avg       0.98      0.98      0.98       880

FrontEnd 

In [ ]:
import pickle

In [ ]:
filename = 'trained_dt_model.sav'
pickle.dump(classifier_dtt, open(filename, 'wb'))

In [ ]:
filename = 'trained_rf_model.sav'
pickle.dump(classifier_rf, open(filename, 'wb'))

In [ ]:
filename = 'trained_lr_model.sav'
pickle.dump(classifier_lr, open(filename, 'wb'))

In [ ]:
filename = 'trained_mlp_model.sav'
pickle.dump(classifier_mlp, open(filename, 'wb'))